In [1]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import *


import numpy as np
import csv
import time
import math
from multiprocessing import Process, Manager

import skopt
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.plots import plot_convergence, plot_objective, plot_evaluations

In [2]:
def loss_min_max(loss):
    if math.isnan(loss):
        loss = 1e+5
    else:
        loss = min(float(loss), 1e+5)
        loss = max(float(loss), 1e-10)
    return loss

In [3]:
def load_pulsar_dataset(adjust_ratio):
    pulsars, stars = [], []
    with open('../../data/chap02/pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            if row[8] == '1': pulsars.append(row)
            else: stars.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8, 1
    
    star_cnt, pulsar_cnt = len(stars), len(pulsars)

    if adjust_ratio:
        data = np.zeros([2*star_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        for n in range(star_cnt):
            data[star_cnt+n] = np.asarray(pulsars[n % pulsar_cnt], dtype='float32')
    else:
        data = np.zeros([star_cnt+pulsar_cnt, 9])
        data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
        data[star_cnt:, :] = np.asarray(pulsars, dtype='float32')
    x = data[:,:8]
    y = data[:,8]
    return x, y

def eval_accuracy(output, y):
    est_yes = np.greater(output, 0.5).reshape(-1,)
    ans_yes = np.greater(y, 0.5).reshape(-1,)
    est_no = np.logical_not(est_yes)
    ans_no = np.logical_not(ans_yes)
    
    tp = np.sum(np.logical_and(est_yes, ans_yes))
    fp = np.sum(np.logical_and(est_yes, ans_no))
    fn = np.sum(np.logical_and(est_no, ans_yes))
    tn = np.sum(np.logical_and(est_no, ans_no))
    
    accuracy = safe_div(tp+tn, tp+tn+fp+fn)
    precision = safe_div(tp, tp+fp)
    recall = safe_div(tp, tp+fn)
    f1 = 2 * safe_div(recall*precision, recall+precision)
    
    return [accuracy, precision, recall, f1]

def safe_div(p, q):
    p, q = float(p), float(q)
    if np.abs(q) < 1.0e-20: return np.sign(p)
    return p / q

def kfold_data(data):
    import sklearn
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=10, shuffle=True)
    kf.get_n_splits(data)
    kfold_data = {}
    for index, (train_index, test_index) in enumerate(kf.split(data)):
        train_data = data[train_index]
        test_data = data[test_index]
        kfold_data[index] = {'train' : train_data, 'test' : test_data}
    return kfold_data

def kfold_return_train(n_fold, kfold_data):
    output_cnt = 8
    if n_fold not in range(10):
        print('{} is not in range(10)'.format(n_fold))
        raise NameError('Change n_fold')
    train_data = kfold_data[n_fold]['train']
    #test_data = kfold_data[n_fold]['test']
    train_data_input = train_data[:, :-output_cnt]
    train_data_output = train_data[:, -output_cnt:]
    return [train_data_input, train_data_output]
def kfold_return_test(n_fold, kfold_data):
    output_cnt = 8
    if n_fold not in range(10):
        print('{} is not in range(10)'.format(n_fold))
        raise NameError('Change n_fold')
    #train_data = kfold_data[n_fold]['train']
    test_data = kfold_data[n_fold]['test']
    test_data_input = test_data[:, :-output_cnt]
    test_data_output = test_data[:, -output_cnt:]
    return [test_data_input, test_data_output]

In [4]:
class pulse_model:
    def __init__(self,learning_rate = 0.01,n_fold = None):
        self.learning_rate = learning_rate
        self.model = self.define_model()
        
    def init_data(self, train_data, test_data):
        self.X_train = train_data[0]
        self.Y_train = train_data[1]
        self.X_test = test_data[0]
        self.Y_test = test_data[1]
        
    def define_model(self,verbose = 0):
        x = Input(shape=(8))
        y = Dense(1, activation='sigmoid')(x)
        __model = Model(x, y)
        if verbose is not 0: __model.summary()
        return __model
    
    def model_compile(self):
        optimizer = tf.keras.optimizers.SGD(learning_rate=self.learning_rate)
        self.model.compile(optimizer=optimizer,
                           loss=tf.keras.losses.BinaryCrossentropy(),
                           metrics=['accuracy']
                          )
    
    def model_fit(self,verbose = 0):
        self.model.fit(x = self.X_train, y = self.Y_train,
                       validation_data = (self.X_test, self.Y_test),
                       batch_size = 64,
                       epochs = 10,
                       verbose = verbose
                      )
        
    def model_evaluate(self,verbose=0):
        output = self.model.predict(self.X_test,verbose=verbose)
        test_loss, test_acc = self.model.evaluate(self.X_test, self.Y_test,verbose=verbose)
        
        result = self.eval_accuracy(output = output, y = self.Y_test)
        #[accuracy, precision, recall, f1]
        
        if verbose is not 0:
            print("acc: {:2.4f}, precis: {:2.4f}, recall: {:2.4f}, f1: {:2.4f}".format(*result))
        
        result_dict = {'acc' : result[0], 'precis' : result[1],
                       'recall' : result[2], 'f1' : result[3],
                       'loss' : test_loss
                      }
        return result_dict
        
        
    def eval_accuracy(self,output, y):
        est_yes = np.greater(output, 0.5).reshape(-1,)
        ans_yes = np.greater(y, 0.5).reshape(-1,)
        est_no = np.logical_not(est_yes)
        ans_no = np.logical_not(ans_yes)

        tp = np.sum(np.logical_and(est_yes, ans_yes))
        fp = np.sum(np.logical_and(est_yes, ans_no))
        fn = np.sum(np.logical_and(est_no, ans_yes))
        tn = np.sum(np.logical_and(est_no, ans_no))

        accuracy = self.safe_div(tp+tn, tp+tn+fp+fn)
        precision = self.safe_div(tp, tp+fp)
        recall = self.safe_div(tp, tp+fn)
        f1 = 2 * self.safe_div(recall*precision, recall+precision)

        return [accuracy, precision, recall, f1]

    def safe_div(self,p, q):
        p, q = float(p), float(q)
        if np.abs(q) < 1.0e-20: return np.sign(p)
        return p / q



In [5]:
class pulse_model_optimize:
    def __init__(self,learning_rate = 0.01,adjust_ratio=False):
        self.learning_rate = learning_rate
        
        
        raw_data = self.load_pulsar_dataset(adjust_ratio)
        self._kfold_data = self.kfold_data(raw_data)
        
        
    def model_trainning(self, train_data, test_data, proc_num=None, return_dict = None):
        model_init = pulse_model(learning_rate=self.learning_rate)
        model_init.init_data(train_data = train_data,
                             test_data = test_data)
        model_init.model_compile()
        model_init.model_fit(verbose=0)
        model_result_dict = model_init.model_evaluate(verbose=0)
        
        if proc_num is not None and return_dict is not None:
            return_dict[proc_num] = model_result_dict

        return model_result_dict
        
    def model_train_multiprocess(self,):
        kfold_data_set = []
        for index in range(10):
            train_data = self.kfold_return_train(n_fold=index, kfold_data=self._kfold_data)
            test_data = self.kfold_return_test(n_fold=index, kfold_data=self._kfold_data)
            kfold_data_set.append((train_data, test_data))
            
        manager = Manager()
        return_dict = manager.dict()
        
        procs = []
        for index, (train_data, test_data) in enumerate(kfold_data_set):
            #print(train_data[0].shape, test_data[0].shape)
            proc = Process(target=self.model_trainning,
                           args=(train_data, test_data,
                                 index, return_dict),
                           name='{}-fold'.format(index)
                          )
            procs.append(proc)
            proc.start()
            
        for proc in procs:
            proc.join()
            
        return return_dict
    
    
    def kfold_model_evaluate_result(self,return_dict, verbose = 0):
        accuracies = []
        precisions = []
        recalls = []
        f1s = []
        losses = []
        
        for index in range(10):
            acc     = return_dict[index]['acc']
            prec    = return_dict[index]['precis']
            recall  = return_dict[index]['recall']
            f1      = return_dict[index]['f1']
            loss    = return_dict[index]['loss']

            accuracies.append(acc)
            precisions.append(prec)
            recalls.append(recall)
            f1s.append(f1)
            losses.append(loss)
            
        mean_acc    = np.array(accuracies).mean()
        mean_prec   = np.array(precisions).mean()
        mean_recall = np.array(recalls).mean()
        mean_f1     = np.array(f1s).mean()
        
        mean_loss = np.array(losses).mean()
        
        return_result_list = [mean_acc, mean_prec, mean_recall, mean_f1, mean_loss]
        #print(return_result_list)
        if verbose is not 0:
            print("acc: {:2.4f}, precis: {:2.4f}, recall: {:2.4f}, f1: {:2.4f}, loss: {:2.4f}".format(*return_result_list))
        

        return return_result_list
        
        
    
    def load_pulsar_dataset(self,adjust_ratio):
        pulsars, stars = [], []
        with open('../../data/chap02/pulsar_stars.csv') as csvfile:
            csvreader = csv.reader(csvfile)
            next(csvreader, None)
            rows = []
            for row in csvreader:
                if row[8] == '1': pulsars.append(row)
                else: stars.append(row)

        #global data, input_cnt, output_cnt
        input_cnt, output_cnt = 8, 1

        star_cnt, pulsar_cnt = len(stars), len(pulsars)

        if adjust_ratio:
            data = np.zeros([2*star_cnt, 9])
            data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
            for n in range(star_cnt):
                data[star_cnt+n] = np.asarray(pulsars[n % pulsar_cnt], dtype='float32')
        else:
            data = np.zeros([star_cnt+pulsar_cnt, 9])
            data[0:star_cnt, :] = np.asarray(stars, dtype='float32')
            data[star_cnt:, :] = np.asarray(pulsars, dtype='float32')
        #x = data[:,:8]
        #y = data[:,8]
        return data
    
    def kfold_data(self, data):
        import sklearn
        from sklearn.model_selection import KFold
        kf = KFold(n_splits=10, shuffle=True)
        kf.get_n_splits(data)
        kfold_data = {}
        for index, (train_index, test_index) in enumerate(kf.split(data)):
            train_data = data[train_index]
            test_data = data[test_index]
            kfold_data[index] = {'train' : train_data, 'test' : test_data}
        return kfold_data

    def kfold_return_train(self, n_fold, kfold_data):
        output_cnt = 1
        if n_fold not in range(10):
            print('{} is not in range(10)'.format(n_fold))
            raise NameError('Change n_fold')
        train_data = kfold_data[n_fold]['train']
        #test_data = kfold_data[n_fold]['test']
        train_data_input = train_data[:, :-output_cnt]
        train_data_output = train_data[:, -output_cnt:]
        return [train_data_input, train_data_output]
    
    def kfold_return_test(self, n_fold, kfold_data):
        output_cnt = 1
        if n_fold not in range(10):
            print('{} is not in range(10)'.format(n_fold))
            raise NameError('Change n_fold')
        #train_data = kfold_data[n_fold]['train']
        test_data = kfold_data[n_fold]['test']
        test_data_input = test_data[:, :-output_cnt]
        test_data_output = test_data[:, -output_cnt:]
        return [test_data_input, test_data_output]

In [6]:
kfold_trainning = pulse_model_optimize(adjust_ratio=True)
result_dict = kfold_trainning.model_train_multiprocess()
result =  kfold_trainning.kfold_model_evaluate_result(result_dict, verbose=1)

acc: 0.8057, precis: 0.8328, recall: 0.8826, f1: 0.8333, loss: 3.6401


In [ ]:
hp_dict = {
    'learning_rate' : 0.05
}

default_HP = list(hp_dict.values())

def model_tunning(hp_list):
    HP_list2dict = {
    'LEARNING_RATE' : float(hp_list[0])
    }
    print("======================start trainning=======================")
    print(HP_list2dict.items())
    print('\n')
    
    tunning_model = pulse_model_optimize(learning_rate = hp_list[0],adjust_ratio=True)
    result_dict = tunning_model.model_train_multiprocess()
    result = tunning_model.kfold_model_evaluate_result(result_dict, verbose=1)
    loss = result[-1]
    

    print('\n')
    print("======================end trainning=======================")
    return loss

#dim_RND_MEAN_nodes = Integer(low=-100, high=100, name='RND_MEAN')
#dim_RND_STD_nodes = Real(low=1e-8, high=1.0, prior='log-uniform', name='RND_STD')
dim_learning_rate_nodes = Real(low=1e-6, high=1.0, prior='log-uniform',name='LEARNING_RATE')

dimension_HP = [
                #dim_RND_MEAN_nodes  ,
                #dim_RND_STD_nodes  ,
                dim_learning_rate_nodes
                ]



n_cell = 30
n_random_starts = 10

gp_fitting = gp_minimize(func=model_tunning,
                        dimensions=dimension_HP,
                        n_calls=n_cell,
                        n_random_starts=n_random_starts,
                        acq_func='EI',
                        x0=default_HP
                        )

In [ ]:
gp_fitting.x

In [7]:
kfold_trainning = pulse_model_optimize(learning_rate=0.0005813962134173598, adjust_ratio=True)
result_dict = kfold_trainning.model_train_multiprocess()
result =  kfold_trainning.kfold_model_evaluate_result(result_dict, verbose=1)

acc: 0.9214, precis: 0.9518, recall: 0.8883, f1: 0.9187, loss: 0.2187


In [8]:
class pulse_model_n_layer(pulse_model):
    def __init__(self,learning_rate=0.01, layer_num = 1, n_unit = 32, n_fold=None):
        self.layer_num = layer_num
        self.n_unit = n_unit
        super().__init__(learning_rate=learning_rate, n_fold=n_fold)
        
        
    def define_model(self, verbose=0):
        x = Input(shape=(8))
        n_layer = x
        for i in range(self.layer_num):
            n_layer = Dense(self.n_unit, activation='relu')(n_layer)
            
        y = Dense(1, activation='sigmoid')(n_layer)
        __model = Model(x,y)
        if verbose is not 0: __model.summary()
        return __model

In [9]:
class pulse_model_n_layer_optimize(pulse_model_optimize):
    def __init__(self, learning_rate=0.01, layer_num = 1, n_unit = 32, adjust_ratio=False):
        self.layer_num = layer_num
        self.n_unit = n_unit
        super().__init__(learning_rate, adjust_ratio)
        
    def model_trainning(self, train_data, test_data, proc_num=None, return_dict = None):
        model_init = pulse_model_n_layer(learning_rate=self.learning_rate,
                                         layer_num=self.layer_num,
                                         n_unit= self.n_unit,
                                        )
        model_init.init_data(train_data = train_data,
                             test_data = test_data)
        model_init.model_compile()
        model_init.model_fit(verbose=0)
        model_result_dict = model_init.model_evaluate(verbose=0)
        
        if proc_num is not None and return_dict is not None:
            return_dict[proc_num] = model_result_dict

        return model_result_dict

In [ ]:
hp_dict = {
    'learning_rate' : 0.05,
    'layer_num' : 1,
    'n_unit' : 32
}

default_HP = list(hp_dict.values())

def model_tunning(hp_list):
    HP_list2dict = {
        'LEARNING_RATE' : float(hp_list[0]),
        'layer_num' : int(hp_list[1]),
        'n_unit' : int(hp_list[2])
    }
    print("======================start trainning=======================")
    print(HP_list2dict.items())
    print('\n')
    
    tunning_model = pulse_model_n_layer_optimize(learning_rate = hp_list[0],
                                                 layer_num=hp_list[1],
                                                 n_unit=hp_list[2],
                                                 adjust_ratio=True
                                                )
    result_dict = tunning_model.model_train_multiprocess()
    result = tunning_model.kfold_model_evaluate_result(result_dict, verbose=1)
    loss = result[-1]
    
    loss = loss_min_max(loss)

    print('\n')
    print("======================end trainning=======================")
    return loss

#dim_RND_MEAN_nodes = Integer(low=-100, high=100, name='RND_MEAN')
#dim_RND_STD_nodes = Real(low=1e-8, high=1.0, prior='log-uniform', name='RND_STD')
dim_learning_rate_nodes = Real(low=1e-6, high=1.0, prior='log-uniform',name='LEARNING_RATE')
dim_layer_num_nodes = Integer(low=0, high=5, name='layer_num')
dim_n_unit_modes = Integer(low=1, high=1024, name='n_unit')

dimension_HP = [
                dim_learning_rate_nodes,
                dim_layer_num_nodes,
                dim_n_unit_modes,
                ]



n_cell = 40
n_random_starts = 10

gp_fitting_n_layer = gp_minimize(func=model_tunning,
                        dimensions=dimension_HP,
                        n_calls=n_cell,
                        n_random_starts=n_random_starts,
                        acq_func='EI',
                        x0=default_HP
                        )

In [ ]:
n_layer_hp = gp_fitting_n_layer.x
n_layer_hp

In [ ]:
n_layer_hp = [6.221358238321396e-05, 2, 717]

In [ ]:
model_test = pulse_model_n_layer(learning_rate=n_layer_hp[0], layer_num=n_layer_hp[1], n_unit=n_layer_hp[2])
X, Y = load_pulsar_dataset(True)
test_train_split = -1000
X_train = X[:test_train_split]
Y_train = Y[:test_train_split]
X_test = X[test_train_split:]
Y_test = Y[test_train_split:]
model_test.init_data(train_data=[X_train, Y_train], test_data=[X_test,Y_test])
model_test.model_compile()
model_test.model_fit(verbose=0)
result = model_test.model_evaluate()

In [ ]:
result

In [10]:
n_layer_hp = [6.221358238321396e-05, 2, 717]
kfold_trainning = pulse_model_n_layer_optimize(learning_rate=n_layer_hp[0], layer_num=n_layer_hp[1], n_unit=n_layer_hp[2],adjust_ratio=True)
result_dict = kfold_trainning.model_train_multiprocess()
result =  kfold_trainning.kfold_model_evaluate_result(result_dict, verbose=1)

acc: 0.9240, precis: 0.9530, recall: 0.8919, f1: 0.9214, loss: 0.2033


In [ ]:
result

In [ ]:
_, = plot_objective(gp_fitting_n_layer)